# 워드 임베딩

- 단어를 벡터로 표현하는 방법
    - 희소표현
    - 밀집표현
    - 워드 임베딩

## 희소표현

- 단어를 원핫인코딩 하게되면 행렬의 값이 대부분 0이고 하나만 1입니다.
- 고로 원핫벡터는 희소 벡터 입니다.
- 원핫벡터는 단어의 의미를 담지 못한다는 단점을 갖고있습니다.

## 밀집표현

- 희소표현과 반대되는 표현입니다.
- 사용자가 밀집 표현의 차원을 128로 설정한다면 모든 단어의 벡터 표현의 차원은 128로 바뀝니다.
    - [0.2, 1.7, ...,-2.2] : 차원이 128


## 워드 임베딩

- 단어를 밀집 벡터의 형태로 표현하는 방법을 워드 임베딩이라고 합니다.
- 밀집 벡터를 워드 임베딩 과정을 통해 나온 결과라고 하여 임베딩 벡터라고도 합니다.
- 방법으로는 LSA, Word2Vec, FastText, Glove 등이 있습니다.
- keras Embedding은 가중치를 학습하는 것과 같은 방식으로 단어 벡터를 학습하는 방법 사용

# 워드투벡터 (Word2Vec)

## 희소표현

- 희소표현으로는 각 단어간 유사성을 표현할 수 없다는 단점이 있었다.
- 이를 위한 대안으로 단어의 의미를 다차원 공간에 벡터화 하는 방법을 찾으려 함
- 이러한 표현 방법을 분산 표현 이라고 합니다.

## 분산표현

- 비슷한 위치에서 등장하는 단어들은 비슷한 의미를 가진다.
- 분산표현은 단어의 의미를 여러 차원에다가 분산하여 표현합니다.
- 이것으로 단어 간 유사도를 계산할 수 있습니다.

## CBOW (Continuous Bag of Words)

- CBOW와 Skip-Gram 방식이 있습니다.
- CBOW
    - 주변에 있는 단어들을 가지고 중간에 있는 단어들을 에측하는 방법
- Skip-Gram
    - 중간에 있는 단어로 주변 단어를 에측하는 방법 입니다.

### CBOW

- 예측해야하는 단어 : 중심단어
- 예측에 사용되는 단어 : 주변단어
- 윈도우
    - 중심 단어를 에측하기 위해서 앞, 뒤로 몇개의 단어를 보는지에 대한 범위
    - ex ) 윈도우의 크기가 2이면 앞에서 두단어 뒤에서 두단어로 중간을 예
- 이 방법을 사용하려면 입력은 모두 원핫 벡터 여야합니다.

### Skip-gram

- 중심 단어로 부터 주변단어를 예측하려고 합니다.


### 네거티브 샘플링(Negative Sampling)

- Word2Vec 모든 단어 집합에 대해서 워드 임베딩을 진행하기 때문에 굉장히 무겁습니다.
- 연관관계가 없는 수많은 단어의 임베딩을 조정할 필요가 없습니다.
- 전체 집합보다 훨씬 작은 단어 집합을 만들어놓고 마지막 단계를 이진분류 문제로 바꿉니다.
- Word2Vec은 주변 단어들을 긍정으로 두고 랜덤으로 샘플링된 단어들을 부정으로 둔 다음에 이진분류를 수행합니다.

## 사전 훈련된 Word2Vec 임베딩

- 영어
    - https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin 파일 경로', binary=True)


- 한국어
    - https://github.com/Kyubyong/wordvectors
    - https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view
    
import gensim
model = gensim.models.Word2Vec.load('ko.bin 파일의 경로')

 Word2vec 모델은 추천 시스템에도 사용되고 있는 모델입니다. 우선 적당하게 데이터를 나열해주면 Word2vec은 위치가 근접한 데이터를 유사도가 높은 벡터를 만들어준다는 점에서 착안된 아이디어입니다.

# 한국어 Word2Vec 해보기

In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [ ]:
train_data = pd.read_csv('../input/naver-movie-review-dataset/ratings_train.txt', sep="\t")

In [ ]:
train_data.tail()

In [ ]:
# 축소해서 진행
train_data = train_data[:30000]

In [ ]:
train_data.tail()

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
stopwords = ['의','가','이','은','는','를','으로','에','와','하다']

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()
tokenized_data = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 # 형태소 추출
    # stem 을 True로 주게되면 ex) 해야지 -> 하다 로 추출
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [ ]:
print('리뷰의 최대 길이 :',max(len(l) for l in tokenized_data))
print('리뷰의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(s) for s in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 0, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

In [ ]:
# 입력한 단어에 대해서 가장 유사한 단어들을 출력하는 model.wv.most_similar
print(model.wv.most_similar("히어로"))

In [ ]:
print(model.wv.most_similar("그녀"))